In [1]:
import torch
import data_utils
from models.DeepAnT import DeepAnT_CNN, DeepAnT_LSTM, Extended_DeepAnT_LSTM
from training.DeepAnT_train import DeepAntTrainingPipeline

In [2]:
# Try to use GPU if available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


# Hyperparameters

In [3]:
# choose between CNN, LSTM, and Extended DeepAnT models
model_type = "Extended"
# choose dataset (kdd99, aapl, gm, axp)
dataset = "kdd99"

In [4]:
if model_type == "DeepAnT_LSTM":
    # Data
    num_features = 7
    seq_length = 1
    seq_stride = 1
    gen_seq_len = seq_length
    batch_size = 8
    # Model
    hidden_dim = 128
    layers = 4
    anm_det_thr = 0.5
    # Training
    num_epochs = 100
    lr=1e-5
    wd=5e-6

In [ ]:
if model_type == "DeepAnT_CNN":
    # Data
    num_features = 34
    seq_length = 30
    seq_stride = 1
    gen_seq_len = 1
    batch_size = 256
    # Model
    dense_dim = 512 
    num_channels = 64
    kernel_size = 1
    anm_det_thr = 0.8
    # Training
    num_epochs = 100
    lr=1e-5
    wd=5e-6

In [ ]:
if model_type == "Extended":
    # Data
    num_features = 6
    seq_length = 50
    seq_stride = 10
    gen_seq_len = 10
    batch_size = 256
    # Model
    hidden_dim = 128
    layers = 4
    anm_det_thr = 0.5
    # Training
    num_epochs = 100
    lr=1e-5
    wd=5e-6

# Load data

In [5]:
if dataset == "kdd99":
    train_dl, test_dl = data_utils.kdd99(seq_length, seq_stride, num_features, gen_seq_len, batch_size, deepant=True)
else:
    file_path = 'data/financial_data/Stocks/' + dataset + '.us.txt'
    tscv_dl_list = data_utils.load_stock_as_crossvalidated_timeseries(file_path, seq_length, seq_stride, gen_seq_len, batch_size, normalise=True)

# Model

In [6]:
if model_type == "DeepAnT_LSTM":
    model = DeepAnT_LSTM(num_features,hidden_dim,layers,anm_det_thr).to(DEVICE)
elif model_type == "DeepAnT_CNN":
    model = DeepAnT_CNN(seq_length,num_features, kernel_size, dense_dim, num_channels, anm_det_thr).to(DEVICE)
elif model_type == "Extended":
    model = Extended_DeepAnT_LSTM(lookback_dim=num_features, hidden_dim=hidden_dim, input_seq_len=seq_length, output_seq_len=gen_seq_len, num_layers=layers, anomaly_threshold=anm_det_thr).to(DEVICE)

In [7]:
loss_function = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(list(model.parameters()), lr=lr, weight_decay=wd)

# Training

In [8]:
pipeline = DeepAntTrainingPipeline()

In [9]:
if dataset == "kdd99":
    pipeline.train_kdd99(train_dl, test_dl, model, optimizer, loss_function, num_epochs, DEVICE)
else:
    pipeline.train(tscv_dl_list, model, optimizer, loss_function, num_epochs, DEVICE)

Epoch 0 loss: 1.3727508263154464
Epoch 1 loss: 1.1622221757065165
Epoch 2 loss: 0.49046601673418827
Epoch 3 loss: 0.36097280085086825
Epoch 4 loss: 0.3523478568277576
Epoch 5 loss: 0.34579538614912464
Epoch 6 loss: 0.34091730734164066
Epoch 7 loss: 0.33766228976574814
Epoch 8 loss: 0.33524822396310894
Epoch 9 loss: 0.3332578285851262
Epoch 10 loss: 0.3315336587754163
Epoch 11 loss: 0.32991697070273485
Epoch 12 loss: 0.3284018376334147
Epoch 13 loss: 0.32708583149042997
Epoch 14 loss: 0.32576057043942536
Epoch 15 loss: 0.3245098475028168
Epoch 16 loss: 0.3231222727759318
Epoch 17 loss: 0.3218234350735491
Epoch 18 loss: 0.3201184670356187
Epoch 19 loss: 0.31776127259839665
Epoch 20 loss: 0.3133134784346277
Epoch 21 loss: 0.306606259603392
Epoch 22 loss: 0.30121533078226176
Epoch 23 loss: 0.29720763408324935
Epoch 24 loss: 0.2933087342164733
Epoch 25 loss: 0.2888105561787432
Epoch 26 loss: 0.2837207044728778
Epoch 27 loss: 0.2780913975428451
Epoch 28 loss: 0.27210783467374067
Epoch 29 los